# Data Analytics

- [x] ETL
- [x] Descriptive/ Explorative
- Diagnostic
- Predictive
- Prescriptive

```
# describe observation inputs
    # - distributions
    # - ranges
    # - line plots
    # - [ time series decomposition ] trend, seasonality etc.
# diagnostics
    # - changes during etl that may affect the data in real world like filling n/a, ffilling cost etc.
# predictive (moving average, exponential smoothing, forecasting)
    # - prediction of future demand/cost based on historical data (relevant/future works ?)
# prescriptive
    # - resulting insights based on predictive analysis, 
    # - impact of choices/results on components later in the pipeline
```

## ETL

In [1]:
import os
import numpy as np
import pandas as pd

### Extract and Save Clean Dataset in a directory

In [2]:
def save_clean_datasets(raw_data_config: dict) -> None:
    
    # pricing
    pricing_data_path = os.path.join(
        raw_data_config["raw_data_dir"], raw_data_config["pricing"]
    )
    os.makedirs(raw_data_config["pricing_save_dir"], exist_ok=True)
    pricing_save_path = os.path.join(
        raw_data_config["pricing_save_dir"], 
        "auction_price.csv"
    )
    pricing_df = pd.read_csv(
        filepath_or_buffer=pricing_data_path,
        usecols=raw_data_config["pricing_cols"],
    )
    pricing_df.rename(
        columns={
            pricing_df.columns[0]: "Timestamp UTC", 
            pricing_df.columns[1]: "auction_price"
        },
        inplace=True,
    )
    pricing_df["Timestamp UTC"] = pd.to_datetime(pricing_df["Timestamp UTC"])
    pricing_df.set_index("Timestamp UTC", inplace=True)
    pricing_df = pricing_df.resample(raw_data_config["index_freq"]).ffill()
    pricing_df[raw_data_config["start_index"]:raw_data_config["end_index"]].to_csv(pricing_save_path)

    # prosumers
    prosumer_data_path1 = os.path.join(
        raw_data_config["raw_data_dir"],
        raw_data_config["customer1"]
    )
    prosumer_data_path2 = os.path.join(
        raw_data_config["raw_data_dir"],
        raw_data_config["customer2"]
    )
    os.makedirs(raw_data_config["prosumer_save_dir"], exist_ok=True)
    prosumer_save_path1 = os.path.join(
        raw_data_config["prosumer_save_dir"],
        "prosumer_1.csv"
    )
    prosumer_save_path2 = os.path.join(
        raw_data_config["prosumer_save_dir"],
        "prosumer_2.csv"
    )
    prosumer_df1 = pd.read_csv(
        filepath_or_buffer=prosumer_data_path1,
        usecols=raw_data_config["customer_cols"]
    )
    prosumer_df2 = pd.read_csv(
        filepath_or_buffer=prosumer_data_path2,
        usecols=raw_data_config["customer_cols"]
    )
    prosumer_df1["Timestamp UTC"] = pd.to_datetime(prosumer_df1["Timestamp UTC"])
    prosumer_df1.set_index("Timestamp UTC", inplace=True)
    prosumer_df2["Timestamp UTC"] = pd.to_datetime(prosumer_df2["Timestamp UTC"])
    prosumer_df2.set_index("Timestamp UTC", inplace=True)

    # save clean datasets
    pricing_df[
        raw_data_config["start_index"]:raw_data_config["end_index"]
    ].to_csv(pricing_save_path)
    prosumer_df1[
        raw_data_config["start_index"]:raw_data_config["end_index"]
    ].to_csv(prosumer_save_path1)
    prosumer_df2[
        raw_data_config["start_index"]:raw_data_config["end_index"]
    ].to_csv(prosumer_save_path2)

    print("Done!")

In [3]:
raw_data_config: dict = {
    "raw_data_dir": "./raw/",
    "pricing_save_dir": "./pricing/",
    "prosumer_save_dir": "./prosumer/",
    "pricing": "entsoe_day_ahead_prices.csv",
    "customer1": "customer_6.csv",
    "customer2": "customer_7.csv",
    "pricing_cols": [0, 3],
    "customer_cols": ["Timestamp UTC", "Power PV", "Power Household", "SoC"],
    "start_index": "2021-08-15 00:00:00+00:00",
    "end_index": "2022-08-15 00:00:00+00:00",
    "index_freq": "15min",
}

save_clean_datasets(raw_data_config)

Done!


### Read CSV files from directory into DataFrames

In [4]:
def get_dataset_as_dataframe(
    directory_path: str, index_col: str, usecols: list, index_freq: str
) -> pd.DataFrame:
    """
    Reads all CSV files in a given directory and returns them as a list of pandas DataFrames.
    Parameters:
        directory_path (str): The path to the directory containing the CSV files.
        index_col (str): The name of the column to be used as the index of the DataFrames.
        usecols (list): A list of column names to include in the DataFrames.
        index_freq (str): The frequency of the index values.
    Returns:
        dataframes (list of pandas.DataFrame): A list of pandas DataFrames, each representing a CSV file in the directory.
    """
    dataframes = []

    try:
        for filename in os.listdir(directory_path):
            if filename.endswith(".csv"):
                try:
                    data_series = pd.read_csv(
                        filepath_or_buffer=os.path.join(directory_path, filename),
                        parse_dates=[index_col],
                        index_col=index_col,
                        usecols=usecols,
                    )
                    data_series.index.freq = index_freq
                    dataframes.append(data_series)
                except Exception as e:
                    print(f"Error reading file {filename}: {e}")
    except FileNotFoundError:
        print(f"Directory {directory_path} not found.")

    return dataframes

In [5]:
data_config = {
    "prosumer_dir": os.path.join("./", "prosumer/"),
    "pricing_dir": os.path.join("./", "pricing/"),
    "index_col": "Timestamp UTC",
    "prosumer_usecols": ["Timestamp UTC", "Power PV", "Power Household", "SoC"],
    "pricing_usecols": ["Timestamp UTC", "auction_price"],
    "index_freq": "15min",
    "train_test_split_ratio": 0.8,
    "save_graphics_index": 24 * 4 * 3,  # 3 days
}

In [6]:
dataframes_prosumer = get_dataset_as_dataframe(
    directory_path=data_config["prosumer_dir"],
    index_col=data_config["index_col"],
    usecols=data_config["prosumer_usecols"],
    index_freq=data_config["index_freq"]
)

dataframes_pricing = get_dataset_as_dataframe(
    directory_path=data_config["pricing_dir"],
    index_col=data_config["index_col"],
    usecols=data_config["pricing_usecols"],
    index_freq=data_config["index_freq"]
)

In [7]:
def merge_transform_dataframes(
    prosumers: pd.DataFrame, pricing: pd.DataFrame, index_col: str
) -> pd.DataFrame:
    """
    Merge and transform two dataframes containing prosumer and pricing data.
    Parameters:
        prosumers (pd.DataFrame): A list of two pandas DataFrames representing prosumer data.
        pricing (pd.DataFrame): A list of two pandas DataFrames representing pricing data.
        index_col (str): The name of the column to be used as the index of the merged DataFrame.
        index_freq (str): The frequency of the index values.
    Returns:
        df (pd.DataFrame): A merged and transformed DataFrame containing the following columns:
            - "Power Household" (np.float32): Power consumption of the household in kWh.
            - "Power PV" (np.float32): Power consumption from PV in kWh.
            - "auction_price" (np.float32): auction price in EUR/kWh.
            - "SoC" (np.float32): State of Charge.
    """
    df = pd.concat(
        [
            pd.merge(prosumers[0], pricing[0], on=index_col, how="left"),
            pd.merge(prosumers[1], pricing[0], on=index_col, how="left"),
        ]
    ).astype(
        {
            "Power Household": np.float32,
            "Power PV": np.float32,
            "auction_price": np.float32,
            "SoC": np.float32,
        }
    )
    # fill na with 0
    df = df.fillna(0)
    # invert power pv sign
    df["Power PV"] = -df["Power PV"]
    # clip power pv
    df["Power PV"] = df["Power PV"].clip(lower=0, upper=df["Power PV"].max(), axis=0)
    # unit conversions
    # power in kwh
    df["Power Household"] /= 1000
    df["Power PV"] /= 1000
    # price in EUR/kWh
    df["auction_price"] /= 1000

    return df

In [8]:
df = merge_transform_dataframes(
    dataframes_prosumer, 
    dataframes_pricing, 
    data_config["index_col"]
)
assert df["auction_price"].iloc[35042] == df["auction_price"].iloc[0]

## Descriptive/ Explorative Analysis

In [9]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

rcParams["figure.figsize"] = 45, 15
rcParams["figure.titlesize"] = "large"
rcParams["figure.labelsize"] = "large"
rcParams["figure.autolayout"] = True
rcParams["font.style"] = "italic"
rcParams["font.variant"] = "normal"
rcParams["font.stretch"] = "condensed"
rcParams["font.family"] = "monospace"
rcParams["font.serif"] = ["Bitstream Vera Sans Mono"]
rcParams["font.size"] = 18
rcParams["grid.linestyle"] = "--"
rcParams["lines.linewidth"] = 3
rcParams["lines.linestyle"] = "-"
rcParams["xtick.major.size"] = 3
rcParams["xtick.minor.size"] = 1
rcParams["xtick.alignment"] = "center"
rcParams["ytick.major.size"] = 3
rcParams["ytick.minor.size"] = 1
rcParams["ytick.alignment"] = "center"
rcParams["legend.frameon"] = False
rcParams["legend.facecolor"] = "inherit"
rcParams["legend.fancybox"] = False
sns.set_style("whitegrid")
sns.set_palette("muted")

In [10]:
plt.hist(df["Power Household"])
plt.title("Power Household Distribution")
plt.savefig(os.path.join("./", "power_household_hist.png"))
plt.close()

In [11]:
plt.hist(df["Power PV"])
plt.title("Power PV Distribution")
plt.savefig(os.path.join("./", "power_pv_hist.png"))
plt.close()

In [12]:
plt.hist(df["auction_price"])
plt.title("Auction Price Distribution")
plt.savefig(os.path.join("./", "auction_price_hist.png"))
plt.close()

In [13]:
#!python -m pip install statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose

In [14]:
decomposed_power_household = seasonal_decompose(
    df["Power Household"][:672*2], # 2 weeks
    model='additive',
    period= 24 * 4  #daily
)

fig = decomposed_power_household.plot()
fig.set_figheight(20)
fig.set_figwidth(40)
fig.set_tight_layout(True)
plt.savefig(os.path.join("./", "power_household_decomposed.png"))
plt.close(fig)

In [15]:
decomposed_power_pv = seasonal_decompose(
    df["Power PV"][:672*2], # 2 weeks
    model='additive',
    period= 24 * 4  #daily
)

fig = decomposed_power_pv.plot()
fig.set_figheight(20)
fig.set_figwidth(40)
fig.set_tight_layout(True)
plt.savefig(os.path.join("./", "power_pv_decomposed.png"))
plt.close(fig)

In [16]:
decomposed_auction_price = seasonal_decompose(
    df["auction_price"][:672*2], # 2 weeks
    model='additive',
    period= 24 * 4  #daily
)

fig = decomposed_auction_price.plot()
fig.set_figheight(20)
fig.set_figwidth(40)
fig.set_tight_layout(True)
plt.savefig(os.path.join("./", "auction_price_decomposed.png"))
plt.close(fig)

## Input Observations

- power demand from household
- power generation from PV
- day ahead auction price to sell/buy energy
- battery initial state of charge

In [17]:
df[["Power Household", "Power PV", "auction_price"]][:500].plot(subplots=True, figsize=(30, 25))
plt.savefig(os.path.join("./", "input_observations_line.png"))
plt.close()

**Flow**

- raw_data --> clean_data(set freq, set time_index range, select columns) and save to appropriate dir (csv).
- merge_transform --> read multiple prosumer and prices, concatenate prosumers and merge with pricing data, missing data filled with 0, invert power pv sign, and appropriate unit conversion.
- load --> load column of dataframe into formats of input observations accepted by the custom envrionment.

___